In [20]:
import yfinance as yf

data = yf.download('AAPL', auto_adjust=True, start='2000-01-01')

fast = 10
slow = 50
data['Cruce'] = data.Close.rolling(fast).mean() / data.Close.rolling(slow).mean() 
data['Señal'] = np.where(data.Cruce > 1.02, 'Compra', np.where(data.Cruce < 0.95, 'Venta', 'Sin Señal'))
data = data.dropna().round(4)

lista = list(zip(data.index, data.Close, data.Señal))
lista

[*********************100%***********************]  1 of 1 downloaded


[(Timestamp('2000-03-14 00:00:00'), 0.88, 'Compra'),
 (Timestamp('2000-03-15 00:00:00'), 0.9, 'Compra'),
 (Timestamp('2000-03-16 00:00:00'), 0.94, 'Compra'),
 (Timestamp('2000-03-17 00:00:00'), 0.96, 'Compra'),
 (Timestamp('2000-03-20 00:00:00'), 0.95, 'Compra'),
 (Timestamp('2000-03-21 00:00:00'), 1.04, 'Compra'),
 (Timestamp('2000-03-22 00:00:00'), 1.11, 'Compra'),
 (Timestamp('2000-03-23 00:00:00'), 1.09, 'Compra'),
 (Timestamp('2000-03-24 00:00:00'), 1.07, 'Compra'),
 (Timestamp('2000-03-27 00:00:00'), 1.08, 'Compra'),
 (Timestamp('2000-03-28 00:00:00'), 1.07, 'Compra'),
 (Timestamp('2000-03-29 00:00:00'), 1.05, 'Compra'),
 (Timestamp('2000-03-30 00:00:00'), 0.97, 'Compra'),
 (Timestamp('2000-03-31 00:00:00'), 1.05, 'Compra'),
 (Timestamp('2000-04-03 00:00:00'), 1.03, 'Compra'),
 (Timestamp('2000-04-04 00:00:00'), 0.98, 'Compra'),
 (Timestamp('2000-04-05 00:00:00'), 1.01, 'Compra'),
 (Timestamp('2000-04-06 00:00:00'), 0.97, 'Compra'),
 (Timestamp('2000-04-07 00:00:00'), 1.02, 'Comp

In [34]:
%%timeit

trades = []
sl =  0.02
i = 0

while i < len(lista):
    if lista[i][2] == 'Compra':
        fecha_in, precio_in = lista[i][0], lista[i][1]
        
        tipo_salida = False
        for j in range(1, len(lista)-i):
            
            #preguntar si salgo por señal
            if lista[i+j][2] == 'Venta':
                tipo_salida = 'Salida x Señal'
                
            #preguntar si salgo por StopLoss
            if  lista[i+j][1] < precio_in * (1-sl) :
                tipo_salida = 'Salida x SL'

            # salgo
            if tipo_salida: 
                fecha_out, precio_out = lista[i+j][0], lista[i+j][1]
                i += j
                trades.append([fecha_in, precio_in,fecha_out, precio_out, tipo_salida])
                break            

    i += 1
    
trades

5.24 ms ± 778 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
import pandas as pd

df = pd.DataFrame(trades, columns=['fecha_in', 'precio_in','fecha_out', 'precio_out', 'tipo_salida'])
df

,fecha_in,precio_in,fecha_out,precio_out,tipo_salida
0,2000-03-14,0.88,2000-05-09,0.81,Salida x SL
1,2000-06-30,0.81,2000-07-24,0.75,Salida x SL
2,2000-07-25,0.77,2000-08-02,0.73,Salida x SL
3,2000-08-29,0.91,2000-09-15,0.85,Salida x SL
4,2000-09-18,0.94,2000-09-21,0.87,Salida x SL
5,2000-09-22,0.81,2000-09-27,0.76,Salida x SL
6,2000-09-28,0.83,2000-09-29,0.40,Salida x SL
7,2001-01-19,0.30,2001-02-20,0.28,Salida x SL
8,2001-02-21,0.29,2001-06-05,0.32,Salida x Señal
9,2001-07-05,0.36,2001-07-06,0.34,Salida x SL


In [10]:
i = 0
while i < 10:
    if i==5:
        i += 3
    
    print(i)
    i +=1 

0
1
2
3
4
8
9
